## Image Narration

This notebook can be used to load an existing trained model, and apply it to images. The vision model is pretrained by FiftyOne, and the natural language generation model is selected from the saved copies of models trained by the Model Trainer notebook.

Note that this notebook must be configured to read from a drive folder, which needs to contain the supporting code found in the repository.

## Approach - Two-Step Pipeline Model

### Imports and setup

In [ ]:
!pip install fiftyone

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.3 MB 11.9 MB/s 
     |████████████████████████████████| 133 kB 46.3 MB/s 
     |████████████████████████████████| 49 kB 1.7 MB/s 
     |████████████████████████████████| 226 kB 20.9 MB/s 
     |████████████████████████████████| 79.9 MB 106 kB/s 
     |████████████████████████████████| 57 kB 668 kB/s 
     |████████████████████████████████| 132 kB 3.9 MB/s 
     |████████████████████████████████| 564 kB 41.3 MB/s 
     |████████████████████████████████| 192 kB 36.1 MB/s 
     |████████████████████████████████| 108 kB 76.4 MB/s 
     |████████████████████████████████| 56 kB 1.6 MB/s 
     |████████████████████████████████| 37.8 MB 1.4 MB/s 
     |████████████████████████████████| 63 kB 925 kB/s 
     |████████████████████████████████| 80 kB 5.0 MB/s 
     |████████████████████████████████| 202 kB 21.0 MB/s 
     |████████████████████████████████| 58

In [ ]:
!pip install torch torchvision

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import fiftyone as fo
import fiftyone.zoo as foz
import torch
import torchvision
from PIL import Image
from torchvision.transforms import functional as func
from fiftyone import ViewField as F
import random

Migrating database to v0.18.0


INFO:fiftyone.migrations.runner:Migrating database to v0.18.0


**Update this section to connect to your google drive:**

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

content_path = '/content/drive/MyDrive/4 - Senior Year/First Semester/COMP 484/Project/CSCAPSTONE'
%cd '/content/drive/MyDrive/4 - Senior Year/First Semester/COMP 484/Project/CSCAPSTONE'
# content_path = 'drive/MyDrive/CSCAPSTONE'
# %cd 'drive/MyDrive/CSCAPSTONE'

# code dependencies (Text.py, LSTM_class.py, etc.) need to be in directory of content_path

Mounted at /content/drive
/content/drive/.shortcut-targets-by-id/1xawm5SPBJkiVQVQP5PI5uwYDs18SeI_N/CSCAPSTONE


### Vision Component

#### FiftyOne data and model

In [ ]:
### grab data
dataset = foz.load_zoo_dataset(
    "coco-2017",
    split="validation",
    dataset_name="detector-recipe",
)

classes = dataset.default_classes

INFO:fiftyone.zoo.datasets:Downloading split 'validation' to '/root/fiftyone/coco-2017/validation' if necessary


INFO:fiftyone.utils.coco:Downloading annotations to '/root/fiftyone/coco-2017/tmp-download/annotations_trainval2017.zip'


 100% |██████|    1.9Gb/1.9Gb [8.9s elapsed, 0s remaining, 214.1Mb/s]      


INFO:eta.core.utils: 100% |██████|    1.9Gb/1.9Gb [8.9s elapsed, 0s remaining, 214.1Mb/s]      


Extracting annotations to '/root/fiftyone/coco-2017/raw/instances_val2017.json'


...

INFO:fiftyone.zoo.datasets:Loading 'coco-2017' split 'validation'


 100% |███████████████| 5000/5000 [39.2s elapsed, 0s remaining, 139.4 samples/s]      


INFO:eta.core.utils: 100% |███████████████| 5000/5000 [39.2s elapsed, 0s remaining, 139.4 samples/s]      


Dataset 'detector-recipe' created


INFO:fiftyone.zoo.datasets:Dataset 'detector-recipe' created


In [ ]:
# Run the model on GPU if it is available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Load a pre-trained Faster R-CNN model
vision_model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
# vision_model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=Trueweights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1)
vision_model.to(device)
vision_model.eval()

print("vision_model ready")

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth


  0%|          | 0.00/160M [00:00<?, ?B/s]

vision_model ready


#### Formatting and interoperability code

In [ ]:
def vision_predict_sample(sample, model):
  """Code decomposed from standard FiftyOne computer vision tutorial. Takes a
  sample from a FiftyOne view, and uses a computer vision model to make a
  prediction of what objects are present in the image."""
  # Load image
  image = Image.open(sample.filepath)
  image = func.to_tensor(image).to(device)
  c, h, w = image.shape
  # Perform inference
  preds = model([image])[0]
  labels = preds["labels"].cpu().detach().numpy()
  scores = preds["scores"].cpu().detach().numpy()
  boxes = preds["boxes"].cpu().detach().numpy()
  # Convert detections to FiftyOne format
  detections = []
  for label, score, box in zip(labels, scores, boxes):
    # Convert to [top-left-x, top-left-y, width, height]
    # in relative coordinates in [0, 1] x [0, 1]
    x1, y1, x2, y2 = box
    rel_box = [x1 / w, y1 / h, (x2 - x1) / w, (y2 - y1) / h]
    detections.append(
      fo.Detection(
        label=classes[label],
        bounding_box=rel_box,
        confidence=score
      )
    )
  # Save predictions to dataset
  sample["predictions"] = fo.Detections(detections=detections)
  sample.save()

In [ ]:
def sample_to_onehot(sample, model, conf_thresh=0.50):
  """Accepts a sample as input, and returns a one-hot vector representing the
  classes detected in that sample image as an output. If predictions have not
  already been made for this sample, then predictions are made.
  Should return a one-hot vector, containing 1s for each class of object
  that was detected. The 0-th position of the vector should always be 0 (this 
  is to match the convention that the class list, `classes`, always reserves
  the index 0)."""
  vec = [0 for i in range(0, len(classes))]
  if not hasattr(sample, "predictions") or sample.predictions is None:
    vision_predict_sample(sample, model)
  predictions = sample.predictions
  for detection in predictions.detections:
    if detection['confidence'] > conf_thresh:
      class_str = detection['label']
      onehot_index = classes.index(class_str)
      vec[onehot_index] = 1
  # TODO: determine whether this information is easier to use as a list or a string
  # one_hot = vec
  one_hot = ''.join(map(str,vec))
  return one_hot

In [ ]:
def generate_onehot(sample, model, conf_thresh=0.5):
  sample["one_hot"] = sample_to_onehot(sample, vision_model)
  sample.save()

In [ ]:
def onehot_to_labels(one_hot, classes):
  labels = []
  for i in range(len(one_hot)):
    if int(one_hot[i]) == 1:
      labels.append(classes[i])
  return labels

### NLG Component

In [ ]:
# These three imports refer to functions in files from the mounted google drive.
import functions as f
from Text import *
from LSTM_class import *

from keras import layers, models, optimizers
import re
import time

#### preprocessing

In [ ]:
# get and prepare training data
path_train = content_path + '/data/train.txt'
input_train = f.read_txt(path_train)

We create two training sets from the same corpus, one containing every word of the corpus in the order they were written, and another containing all of the words of the corpus in reverse order.

In [ ]:
max_len = 4
step = 3

text_train_forward = Text(input_train, reverse=False)
text_train_reverse = Text(input_train, reverse=True)
text_train_forward.tokens_info()

seq_train_forward = Sequences(text_train_forward, max_len, step)
seq_train_reverse = Sequences(text_train_reverse, max_len, step)
seq_train_forward.sequences_info()

total tokens: 1428900, distinct tokens: 42415
number of sequences of length 4: 476299


Here are some examples of sequences in the corpus:

In [ ]:
print(text_train_forward.tokens[:10])
print(text_train_forward.tokens_ind[:10], '\n')
np.array(seq_train_forward.sequences[:3])

['I', 'entered', 'this', 'incarnation', 'on', 'March', 'the', 'twenty', '-', 'ninth']
[8273, 13751, 37818, 19540, 2093, 22253, 7033, 26673, 36328, 31989] 



array([[ 8273, 13751, 37818, 19540],
       [19540,  2093, 22253,  7033],
       [ 7033, 26673, 36328, 31989]])

The reverse sequences are not necessarily exact reverses of the forward sequences because the total number of tokens in the corpus doesn't necessariy divide evenly into 4-word subsequences, so one to three words may be left off of the end.

In [ ]:
print(text_train_reverse.tokens[-10:])
print(text_train_reverse.tokens_ind[-10:], '\n')
np.array(seq_train_reverse.sequences[-3:])

['ninth', '-', 'twenty', 'the', 'March', 'on', 'incarnation', 'this', 'entered', 'I']
[31989, 36328, 26673, 7034, 22253, 2093, 19540, 37818, 13751, 8274] 



array([[19677,  9512, 31989, 36328],
       [36328, 26673,  7034, 22253],
       [22253,  2093, 19540, 37818]])

In order for the model to work well, we want to ensure that at least most of the image keywords that we'll be asking it to narrate are present in the vocabulary of the natural language model. This code tells us which keywords are present and which are missing:

In [ ]:
def keyword_in_corpus(keyword, corpus=text_train_forward):
  subwords = keyword.split(' ')  # some COCO keywords are actually two words
  # TODO: two digit numbers should also be considered as two individual digits?
  flag = True
  for subword in subwords:
    flag = flag and subword in corpus.token2ind.keys()
  return flag

def validate_corpus(corpus):
  """Returns a list of any tokens which might be detected in an image by the
  vision model, but which are not in the vocabulary of this corpus. Ideally,
  this list should only contain the number 0."""
  missing_vocab = []
  present_vocab = []
  for word in classes:
    l = present_vocab if keyword_in_corpus(word, corpus) else missing_vocab
    l.append(word)
  return {'missing':missing_vocab, 'present':present_vocab}

In [ ]:
attendance = validate_corpus(text_train_forward)
print("corpus contains", len(attendance['present']), "MSCOCO keywords, out of", len(classes), "--- this is about", int(10000*(len(attendance['present'])/len(classes)))/100, "% attendance")

corpus contains 70 MSCOCO keywords, out of 91 --- this is about 76.92 % attendance


In [ ]:
print(attendance['missing'])

['motorcycle', 'airplane', 'fire hydrant', 'zebra', 'giraffe', 'backpack', 'frisbee', 'skis', 'snowboard', 'kite', 'skateboard', 'surfboard', 'broccoli', 'pizza', 'donut', 'tv', 'laptop', 'keyboard', 'microwave', 'toaster', 'teddy bear']


We're missing a few modern words, but we still have 70 out of 91 keywords available, so the language model should have plenty to say for every image, even if it doesn't recognize a few keywords here and there.

#### Loading natural language model

Here we use keras to load an existing natural language model. We also need to lean on a bit of json, because even with the same corpus, different models sometimes encode text differently. For example, one model might associate the word token "person" with the index `18382`, while another model might associate "person" with index `918`. In order to reuse a model, we have to know how it maps words to these indices, and that's what we store as json.

In [ ]:
import json

In [ ]:
target_nano = "1670852439439527500"  # chooses a particular version of the trained models, based on the nano time that the forward model finished training
# this particular nano corresponds to a model which was trained for 90 minutes.
# loads the version of the token--index mappings that this particular model was trained on
version_path = content_path + "/out/model_" + str(target_nano) + "/"
token2ind_forward = json.load(open(version_path + "word_mapping_forward.json"))
ind2token_forward = {value: key for (key, value) in token2ind_forward.items()}
map_forward = (token2ind_forward, ind2token_forward)
token2ind_reverse = json.load(open(version_path + "word_mapping_reverse.json"))
ind2token_reverse = {value: key for (key, value) in token2ind_reverse.items()}
map_reverse = (token2ind_reverse, ind2token_reverse)
model_nlg_forward = models.load_model(version_path + "model_nlg_forward")
model_nlg_reverse = models.load_model(version_path + "model_nlg_reverse")

#### Natural language generation code

This is the code which performs the basic NLG tasks required by the model. This doesn't interact with or account for images in any way, this code only accomplishes the step of generating sentences from keywords.

In [ ]:
def produce_reversed_prediction(pred, text):
  r0 = text.split(' ')
  r0.reverse()
  r0 = ' '.join(r0)
  return pred.reverse_preprocess(r0)

In [ ]:
def generate_reversed_prediction(pred, length=100, temperature=1):
  text = pred.generate_sequence(length, temperature=temperature, ret_unprocessed=True)
  return produce_reversed_prediction(pred, text)

In [ ]:
def split_sentences(string):
  """Splits an input containing multiple sentences into a list of seperate
  sentences, including the original punctuation."""
  # please god do not look at this dispicable regex
  delimiters = '.!?'
  l = []
  l.append([])
  for c in string:
    l[-1].append(c)
    if c in delimiters:
      l.append([])
  l2 = []
  for slist in l:
    s_l = ''.join(slist)
    if not (s_l == '' or s_l == ' '):  # iffy attempt to address trailing whitespace
      l2.append(s_l.strip())
  return l2

In [ ]:
def generate_descriptive_sentence(keyword, seq_len=100, temperature=1):
  """Given an input keyword contained in the corpus, generates a single sentence
  of first-person English natural language which includes that keyword.
  `seq_len` determines the maximum length of the subsequences generated when
  generating the full target sentence. This is not necessarily the length of the
  resulting sentence---typically, that sentence will be shorter, because only
  one complete sentence from the final output is returned. The final output
  will be no more than `2 * seq_len` tokens long (where our tokens are words and
  punctuation). Low `seq_len` can occasionally result in truncated sentences,
  wherein the model did not generate a full sentence before it reached the token
  limit. High `seq_len`, however, is more computationally expensive for equal
  outputs, because most of the additional tokens are not kept in the final
  output. For this reason, a sequence length of around 50-150 is recommended.
  `temperature` is a randomization factor between 0 and 1. The higher the
  temperature, the more likely the model is to use unusual words from its corpus
  (essentially, the more variety there will be in the output). Lower temperature
  results in very stable and more gramatically correct language, but much less
  variation and 'color' in the output."""
  # known issues:
  #    sometimes produces really long sentences. Maybe reduce seq_len?
  #    this may sometimes falsely cut sentence borders at Mr. or Ms. and other such abbreviations that include periods
  # 
  # if the keyword is actually two or more words, such as "wine glass"
  # then reverse it, so it comes out correctly 
  reverse_keyword = keyword if len(keyword.split(' ')) == 1 else ' '.join(reversed(keyword.split(' ')))
  pred_reverse = gen_pred(reverse_keyword, model_nlg_reverse, *map_reverse)
  reverse_prediction = generate_reversed_prediction(pred_reverse, seq_len, temperature)
  last_sentence_of_reverse = split_sentences(reverse_prediction)[-1]
  pred_forward = gen_pred(last_sentence_of_reverse, model_nlg_forward, *map_forward)
  forward_prediction = pred_forward.generate_sequence(seq_len, temperature=temperature)
  first_sentence_of_forward = split_sentences(forward_prediction)[0]
  # adds punctuation if the sentence was cut off early
  if first_sentence_of_forward[-1] not in ".!?":
    first_sentence_of_forward += "."
  return first_sentence_of_forward


def gen_pred(prefix, model, token2ind, ind2token):
  """Convenience function that wraps the process of making a ModelPredict
  object."""
  text_prefix= Text(prefix, token2ind, ind2token)
  return ModelPredict(model, text_prefix, token2ind, ind2token, max_len, embedding=True)

### Synthesis

#### Synthesis code

This code interoperates between the vision model and the natural language model, and provides functions for producing natural language paragraphs from input images. The main function here is `narrate_images()`

In [ ]:
def find_classes_in(samples_view, vision_model, conf_thresh=0.50):
  class_detections = []
  for sample in samples_view:
    generate_onehot(sample, vision_model, conf_thresh)
    labels = onehot_to_labels(sample['one_hot'], classes)
    for label in labels:  # but appending individual labels lets us easily filter out to only unique labels
      class_detections.append(label)
  return tuple(set(class_detections))  # removes duplicate labels

In [ ]:
def generate_sentences_from(class_detections, temperature=0.7, max_sentences=3, seed=None):
  # randomly reorders the keywords, so their corresponding sentences don't just
  # appear in alphabetical order for every prediction.
  if seed is None:
    keywords = random.sample(class_detections, len(class_detections))
  else:
    random.seed(seed)
    keywords = random.sample(class_detections, len(class_detections))
  # only generates up to `max_sentences` sentences. This means that, in cases
  # where there are many detections, some may be ignored. This process could
  # be improved by, perhaps, prioritizing high confidence detections over low
  # confidence detections, when deciding which to include and which to exclude.
  sentences = []
  for keyword in keywords:
    if len(sentences) < max_sentences and keyword_in_corpus(keyword):
      sentences.append(generate_descriptive_sentence(keyword, temperature=temperature))
  return sentences

In [ ]:
def narrate_images(samples_view, conf_thresh=0.5, temperature=0.7, max_sentences=3, seed=None):
  """Given a set of input images, generates first-person narrative text based on
  the detected contents of those images.
  `conf_thresh` determines how confident the vision model must be in a detection
  in order for it to be included in the list of classes which the natural
  language model generates sentences for. If this is very high, some images
  might not result in any output.
  `temperature` determines how much the natural language model will deviate from
  the most likely word selection. If this value is high, then the generate text
  will almost always be gramatically correct, but is likely to be repetitive. If
  the temperature is low, then the output will be much more varied, but it may
  become ungrammatical, and is more likely to be nonsensical.
  `max_sentences` is the maximum number of complete sentences which the natural
  language model will attempt generate from the given set of images. If this is
  high, the output could be very long, but will almost certainly include a
  sentence for every detected class of object. If this value is low, the output
  will be shorter and more concise, but may leave out some of the detections.
  `seed` is a seed used for random processes, namely the random reordering of
  keywords. This won't cause the same text to be generated on subsequent calls
  (randomness is baked into the natural language model), but it will cause the
  class detections to be presented in the same order each time."""
  class_detections = find_classes_in(samples_view, vision_model, conf_thresh)
  sentences = generate_sentences_from(class_detections, temperature, max_sentences, seed)
  paragraph = ' '.join(sentences)
  return paragraph

In [ ]:
def narrate_n(num_images, conf_thresh=0.5, temperature=0.7, max_sentences=3, seed=None):
  """Chooses `num_images` images at random from the currently selected FiftyOne
  dataset, and uses them to generate first-person narrative text."""
  samples_view = dataset.take(num_images) if seed is None else dataset.take(num_images, seed)
  paragraph = narrate_images(samples_view, conf_thresh, temperature, max_sentences, seed)
  return paragraph

## Execution

The following code selects three images from the MSCOCO 2017 dataset, and then applies our code to generate a first-person narrative about the contents of the image. A different set of images can be selected by changing the `seed` variable.

In [ ]:
seed = 105

In [ ]:
samples_view = dataset.take(3, seed=seed)
session = fo.launch_app(samples_view)

In [ ]:
# narrate_images(samples_view, seed=seed)

In [ ]:
for i in range(3):
  print(narrate_images(samples_view, seed=seed))
  print()

I saw him, he found it was left, and gave it a story of the dog. I went back to my hold of my time, knowing was, and that the college, but when they came back to me, I suppose, but it were well of all of of the business to my wife, who, her husband and the daughter, a note, forty- five days later in a person, mainly of the-- sage attendants of she was a great a good man. I stood in the window, I looked at him, and straight, and went to the car, and I would have the same room for the night.

One of the day or the present it was the dog and stopped the General was the one, as the old age of the-- personal son was was was up. I gave him not a person to take the place, would not school, I was after the third know it was the first time in the number of my first strict and; that the right of the house is not going to the very men who is the where they lived in the house that of their personal intelligence. He came out of the car of the day, being a comfortable and a man.

Yet him all the dog